<a href="https://colab.research.google.com/github/secutron/TesTime/blob/main/FixMatch_ignite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install requirements:

In [2]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'


try:
  import torch_xla
  import ignite
except ImportError:
  # VERSION = "nightly"
  # VERSION = "20200607"
  VERSION = "2020060f"

  !pip install gsutil

  !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
  !python pytorch-xla-env-setup.py --version $VERSION
  !pip install --upgrade git+https://github.com/pytorch/ignite
  !pip install --upgrade --pre hydra-core
  !pip install wandb
  !pip uninstall -y pillow && CC="cc -mavx2" pip install --no-cache-dir --force-reinstall pillow-simd


     |████████████████████████████████| 2.9MB 6.7MB/s 
     |████████████████████████████████| 174kB 40.0MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 1.4MB 40.2MB/s 
     |████████████████████████████████| 112kB 41.1MB/s 
     |████████████████████████████████| 3.2MB 44.7MB/s 
  Created wheel for gsutil: filename=gsutil-4.61-cp37-none-any.whl size=3709861 sha256=83f47cc8da67254ff6c84c881d0ea87f85883bbf79f5b9d4046ded0b67d494ae
  Stored in directory: /root/.cache/pip/wheels/21/c8/4c/b30f998197ac91cb58cbc8b63ff3654724f080391131fde96c
  Created wheel for gcs-oauth2-boto-plugin: filename=gcs_oauth2_boto_plugin-2.7-cp37-none-any.whl size=23204 sha256=80a9a958b1f3afa016eeddf4c1457cc4bee10c3c0e221847c6371e0e95247789
  Stored in directory: /root/.cache/pip/wheels/2e/6b/7c/bd86832ceb17e0ae3d362c44f461832452eeaacddfcf9128ee
  Created w

     |████████████████████████████████| 2.1MB 5.1MB/s 
     |████████████████████████████████| 133kB 50.4MB/s 
     |████████████████████████████████| 163kB 52.3MB/s 
     |████████████████████████████████| 102kB 10.1MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=c7a298a84b3bfda09d77dc9805577651212ac55b02c352a6aa87b1e682b5250b
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=9c9a7e34c2c25a8704a74a3403198a97574eb133878541ff8a38bdc9836ae2ef
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
Uninstalling Pillow-7.1.2:
  Successfully uninstalled Pillow-7.1.2
     |████████████████████████████████| 634kB 4.9MB/s 
  Created wheel for pillow-

### Download dataset:

In [3]:
# Download dataset:
from torchvision.datasets import CIFAR10

CIFAR10("/tmp/cifar10", train=True, download=True);


Extracting /tmp/cifar10/cifar-10-python.tar.gz to /tmp/cifar10


### Get the code:

In [4]:
!git clone https://github.com/vfdev-5/FixMatch-pytorch/

Cloning into 'FixMatch-pytorch'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 210 (delta 121), reused 164 (delta 76), pack-reused 0
Receiving objects: 100% (210/210), 72.05 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (121/121), done.


#### Optionally, login to `W&B`

To skip logging to `W&B`, please set `online_exp_tracking.wandb=false` 

In [ ]:
# !wandb login <token>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


### Let's train ResNet18 model in a faster mode

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python main_fixmatch.py distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true solver.num_epochs=50 \
    ssl.confidence_threshold=0.7 ema_decay=0.9 ssl.cta_update_every=15 solver.optimizer.params.lr=0.1

2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'xla-tpu'
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
2020-06-15 14:45:26,154 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f7b29cdbd90>' in 8 processes
2020-06-15 14:46:42,829 FixMatch Training INFO: name: fixmatch
seed: 543
debug: false
model: resnet18
num_classes: 10
ema_decay: 0.9
solver:
  unsupervised_criterion:
    cls: torch.nn.CrossEntropyLoss
    params:
      reduction: none
  num_epochs: 50
  epoch_length: 128
  checkpoint_every: 500
  validate_every: 1
  resume_from: null
  optimizer:
    cls: torch.optim.SGD
    params:
      lr: 0.1
      momentum: 0.9
      weight_decay: 0.0001
      nesterov: false
  supervised_criterion:
    cls: torch.nn.CrossEntropyLoss
  lr_scheduler:
    cls: torch.optim.lr_scheduler.CosineAnnealing

### Let's train WRN-28-2 model

In [ ]:
!cd FixMatch-pytorch && export PYTHONPATH=$PWD:$PYTHONPATH && \
  python -u main_fixmatch.py model=WRN-28-2 distributed.backend=xla-tpu distributed.nproc_per_node=8 online_exp_tracking.wandb=true